In [1]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_30000_samples.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16,pin_memory=True)
num_classes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


5

In [2]:
train_dataset[0][1][0]

tensor(3)

In [3]:
from src.model import PoseWithClassModel
from torch.optim.lr_scheduler import StepLR
model = PoseWithClassModel(num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75) 


In [4]:
from src.train_utils import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.is_available()
torch.backends.cudnn.version()
torch.cuda.is_available()
torch.cuda.get_device_properties(0)
print(f"Using device: {device}")


Using device: cuda


In [5]:
torch.backends.cudnn.is_available()

True

In [6]:
torch.backends.cudnn.version()

90100

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4070', major=8, minor=9, total_memory=12281MB, multi_processor_count=46, uuid=9e60166a-e60f-451d-3cb0-2063affabbf5, L2_cache_size=36MB)

In [9]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=100)

Training: 100%|██████████| 188/188 [03:21<00:00,  1.07s/it]


Epoch 1/100 - Total Loss: 1.8444 - Cls: 1.2625 - Trans: 0.0544 - Rot: 0.6301


Validating: 100%|██████████| 47/47 [00:45<00:00,  1.03it/s]


Validation - Loss: 3.8496 | Cls Loss: 3.6377 | Trans Loss: 0.1035 | Rot Loss: 0.5773 | Accuracy: 22.82%


Training: 100%|██████████| 188/188 [03:20<00:00,  1.07s/it]


...

Validation - Loss: -10.6380 | Cls Loss: 0.0001 | Trans Loss: 0.0015 | Rot Loss: 0.4657 | Accuracy: 100.00%


Training: 100%|██████████| 188/188 [03:20<00:00,  1.07s/it]


Epoch 100/100 - Total Loss: -9.1872 - Cls: 0.7049 - Trans: 1.0262 - Rot: 1.0074


Validating: 100%|██████████| 47/47 [00:44<00:00,  1.05it/s]

Validation - Loss: -7.9882 | Cls Loss: 0.0052 | Trans Loss: 0.0043 | Rot Loss: 0.4862 | Accuracy: 99.80%


In [ ]:
one_pcd = dataset[2][0]

model.eval()
with torch.no_grad():
    # Move the point cloud to the same device as the model
    one_pcd = one_pcd.to(device)
    # Forward pass through the model
    # Note: Add batch dimension
    class_logits, pred_pose = model(one_pcd.unsqueeze(0).to(device))
one_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(one_pcd.cpu().numpy()))
one_pcd.paint_uniform_color([0.5, 0.5, 0.5])

PointCloud with 1000 points.

In [ ]:
from pathlib import Path
import open3d as o3d
import numpy as np
model_dir = Path("./data/models")
model_list = list(model_dir.glob("*"))

def check_model_on_instance(model, dataset, idx, device, paths):
    pcd = dataset[idx][0]
    gt_class = dataset[idx][1][0]
    gt_rotation_quaternion = dataset[idx][1][1]
    gt_translation = dataset[idx][1][2]
    gt_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd))
    
    model.eval()
    with torch.no_grad():
        # Move the point cloud to the same device as the model
        pcd = pcd.to(device)
        # Forward pass through the model
        # Note: Add batch dimension
        class_logits, pred_pose = model(pcd.unsqueeze(0).to(device))
        preds = class_logits.argmax(dim=1)
        model_path = paths[preds.item()]
        print(f"Predicted class: {preds.item()}", 
              f"GT class: {gt_class.item()}")
        print(f"Model path: {model_path}")
    pcd_predicted = o3d.io.read_triangle_mesh(model_path)
    pcd_predicted.paint_uniform_color([1, 0, 0])  # Red color for predicted model
    predicted_quaternion = pred_pose[0, 3:].cpu().numpy()

    q1, q2, q3, q4 = predicted_quaternion/np.linalg.norm(predicted_quaternion)
    print(np.linalg.norm(np.array([q1, q2, q3, q4])))
    x, y, z = pred_pose[0, :3].cpu().numpy()
    # Convert quaternion to rotation matrix
    T = np.array([
        [1 - 2*(q3**2 + q4**2),     2*(q2*q3 - q1*q4),
            2*(q2*q4 + q1*q3), x],
        [2*(q2*q3 + q1*q4), 1 - 2*(q2**2 + q4**2),     2*(q3*q4 - q1*q2), y],
        [2*(q2*q4 - q1*q3),     2*(q3*q4 + q1*q2), 1 - 2*(q2**2 + q3**2), z],
        [0, 0, 0, 1]
    ])
    pcd_predicted.transform(T)
    translation_diff = np.array([x, y, z]) - gt_translation.numpy()
    print(f"Translation diff: {translation_diff}")
    translation_error = sum((np.array([x, y, z]) - gt_translation.cpu().numpy())**2)/3
    print(f"Translation error: {translation_error}")
    rotation_error = 1 - np.dot(predicted_quaternion, gt_rotation_quaternion)
    print(f"Rotation error: {rotation_error}")
    # Visualize the point cloud and the predicted model
    o3d.visualization.draw_geometries([pcd_predicted, gt_pcd])


In [ ]:
check_model_on_instance(model, train_dataset, 0, device, model_list)

Predicted class: 2 GT class: 2
Model path: data\models\hammer.obj
0.99999994
Translation diff: [ 0.14558786 -0.05423252  0.06600547]
Translation error: 0.00949790452917417
Rotation error: -0.47630763053894043


In [ ]:
preds = class_logits.argmax(dim=1)
preds.item()

3

In [ ]:



model_path = model_list[preds.item()]

In [ ]:
q1, q2, q3, q4 = pred_pose[0, 3:].cpu().numpy()
print(q1, q2, q3, q4)

0.10637765 -2.4367216 1.1922573 6.49378


In [ ]:

pcd_predicted = o3d.io.read_triangle_mesh(model_path)
pcd_predicted.paint_uniform_color([1, 0, 0])  # Red color for predicted model
translation = pred_pose[0, :3].cpu().numpy()
rotation = pred_pose[0, 3:].cpu().numpy()
pcd_predicted.rotate(pcd_predicted.get_rotation_matrix_from_quaternion(rotation))
pcd_predicted.translate(translation)

pcd_predicted

TriangleMesh with 746 points and 1476 triangles.

In [ ]:
((dataset[2][1][2] - translation)**2).sum()/3

tensor(0.0035)

In [ ]:
import numpy as np
print(dataset[2][1][1], rotation/np.linalg.norm(rotation))
1-np.dot(dataset[2][1][1], rotation/np.linalg.norm(rotation))

tensor([-0.3478, -0.0184, -0.8122,  0.4679]) [ 0.01511382 -0.3462021   0.16939235  0.92261684]


0.7047482132911682

In [ ]:
np.linalg.norm(dataset[1][1][1])

1.0

In [ ]:
o3d.visualization.draw_geometries([pcd_predicted, one_pcd])